In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [96]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [97]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [98]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [100]:
# Path to the CSV file (adjust the path based on where your file is in Google Drive)
file_path = '/content/drive/My Drive/data/training_data.csv'

# Read the CSV file into a DataFrame
news_df = pd.read_csv(file_path)


In [101]:
news_df.shape

(120000, 2)

In [102]:
news_df.columns

Index(['text', 'label'], dtype='object')

In [103]:
news_df.head()

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2


In [104]:
news_df.tail()

,text,label
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1
119999,Nets get Carter from Raptors INDIANAPOLIS -- A...,1


In [105]:
news_df.isnull().sum()

,0
text,0
label,0


In [106]:
news_df["label"].value_counts() # pefectly balanced class

,count
label,
2,30000
3,30000
1,30000
0,30000


# Text Processing

In [107]:
# Splitting data into train and validation
from sklearn.model_selection import train_test_split

X = news_df["text"]
y = news_df["label"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = news_df["label"])

In [108]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(96000,)
(24000,)
(96000,)
(24000,)


In [109]:
# Converting Series to dataframe
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

In [110]:
pd.DataFrame(y_train)["label"].value_counts()

,count
label,
1,24000
3,24000
0,24000
2,24000


In [111]:
pd.DataFrame(y_val)["label"].value_counts()

,count
label,
2,6000
1,6000
0,6000
3,6000


In [112]:
def preprocess_text(text):
  # converting to lowercase
  text = text.lower()

  # removing punctuations, digits and special characters
  text = re.sub(r'[^a-zA-Z\s]', '', text)

  # removing stop words
  stop_words = set(stopwords.words('english'))
  text = ' '.join([w for w in text.split() if w not in stop_words])

  # tokenization
  tokens = word_tokenize(text)

  # lemmatization
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(w) for w in tokens]

  # remove short tokens
  tokens = [w for w in tokens if len(w) > 2]

  # Joining the text back to cleaned sentence
  clean_text = ' '.join(tokens)

  return clean_text

In [113]:
X_train["processed_text"] = X_train["text"].apply(preprocess_text)

In [114]:
X_val["processed_text"] = X_val["text"].apply(preprocess_text)

In [115]:
X_train.shape, X_val.shape

((96000, 2), (24000, 2))

In [116]:
X_train.head()

,text,processed_text
51238,"Clijsters Unsure About Latest Injury, Says Hew...",clijsters unsure latest injury say hewitt toky...
32395,Earthquake Swarm Shakes Eastern Sierra (AP) AP...,earthquake swarm shake eastern sierra swarm ea...
54318,Issues related to war in Iraq help define elec...,issue related war iraq help define election au...
108419,NCAA probation for little engine that could RO...,ncaa probation little engine could romeoville ...
419,eBay Buys 25 of CraigList eBay Buys 25 of crai...,ebay buy craiglist ebay buy craiglistebay inc ...


# Adding Semantic Polarity

Different news topics have different polarities, for example, sports is often positive, world news is more neutral/negative

In [117]:
from textblob import TextBlob

In [118]:
X_train['sentiment'] = X_train['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
X_val['sentiment'] = X_val['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Capturing Stylistic or Structural Cues

usually business headlines are longer than sports

In [119]:
X_train['text_len'] = X_train['text'].apply(lambda x: len(x.split()))
X_val['text_len'] = X_val['text'].apply(lambda x: len(x.split()))

In [120]:
X_train.head()

,text,processed_text,sentiment,text_len
51238,"Clijsters Unsure About Latest Injury, Says Hew...",clijsters unsure latest injury say hewitt toky...,0.375000,35
32395,Earthquake Swarm Shakes Eastern Sierra (AP) AP...,earthquake swarm shake eastern sierra swarm ea...,-0.100000,32
54318,Issues related to war in Iraq help define elec...,issue related war iraq help define election au...,0.200000,33
108419,NCAA probation for little engine that could RO...,ncaa probation little engine could romeoville ...,-0.093750,31
419,eBay Buys 25 of CraigList eBay Buys 25 of crai...,ebay buy craiglist ebay buy craiglistebay inc ...,0.283333,56


# Standarizing numeric features

In [121]:
from sklearn.preprocessing import StandardScaler

In [122]:
num_features = ["sentiment", "text_len"]
scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])

In [123]:
X_train.head()

,text,processed_text,sentiment,text_len
51238,"Clijsters Unsure About Latest Injury, Says Hew...",clijsters unsure latest injury say hewitt toky...,1.587942,-0.283384
32395,Earthquake Swarm Shakes Eastern Sierra (AP) AP...,earthquake swarm shake eastern sierra swarm ea...,-0.833072,-0.579694
54318,Issues related to war in Iraq help define elec...,issue related war iraq help define election au...,0.695989,-0.480924
108419,NCAA probation for little engine that could RO...,ncaa probation little engine could romeoville ...,-0.801217,-0.678463
419,eBay Buys 25 of CraigList eBay Buys 25 of crai...,ebay buy craiglist ebay buy craiglistebay inc ...,1.120729,1.790783


In [124]:
X_val[num_features] = scaler.transform(X_val[num_features])

In [125]:
X_val.head()

,text,processed_text,sentiment,text_len
112486,Wall Street's Designs on '05? A Merger Boom Af...,wall street design merger boom year selfdoubt ...,-0.323385,-0.678463
109163,Dodgers sign Ledee to two-year deal The Los An...,dodger sign ledee twoyear deal los angeles dod...,2.225051,-0.876003
6638,Olympians pursuit of marketing gold begins lon...,olympian pursuit marketing gold begin long med...,0.632279,0.506775
3996,New Device: Flying Robot Seiko Epson hopes the...,new device flying robot seiko epson hope tiny ...,0.661238,0.210465
39522,Spanish laureate 'was informer' Prize-winning ...,spanish laureate informer prizewinning spanish...,-0.238437,-1.962472


# Applying TF-IDF vectorization on processed text

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
t = vectorizer.fit_transform(X_train['processed_text'])

In [127]:
type(t)

scipy.sparse._csr.csr_matrix

In [128]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [129]:
preprocessor = ColumnTransformer([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1,2)), 'processed_text'),
    ('num', StandardScaler(), ['sentiment', 'text_len'])
])

In [130]:
pipeline = Pipeline([
    ('features', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

In [131]:
pipeline.fit(X_train[['processed_text', 'sentiment', 'text_len']], y_train)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('tfidf',
                                                  TfidfVectorizer(max_features=5000,
                                                                  ngram_range=(1,
                                                                               2)),
                                                  'processed_text'),
                                                 ('num', StandardScaler(),
                                                  ['sentiment', 'text_len'])])),
                ('clf', LogisticRegression(max_iter=1000))])

In [132]:
X_val

,text,processed_text,sentiment,text_len
112486,Wall Street's Designs on '05? A Merger Boom Af...,wall street design merger boom year selfdoubt ...,-0.323385,-0.678463
109163,Dodgers sign Ledee to two-year deal The Los An...,dodger sign ledee twoyear deal los angeles dod...,2.225051,-0.876003
6638,Olympians pursuit of marketing gold begins lon...,olympian pursuit marketing gold begin long med...,0.632279,0.506775
3996,New Device: Flying Robot Seiko Epson hopes the...,new device flying robot seiko epson hope tiny ...,0.661238,0.210465
39522,Spanish laureate 'was informer' Prize-winning ...,spanish laureate informer prizewinning spanish...,-0.238437,-1.962472
...,...,...,...,...
105468,"Let the Bidding Begin Wasting little time, Oma...",let bidding begin wasting little time omar min...,-0.801217,-1.271083
90241,Babson finishes strong to reach NCAA tourney F...,babson finish strong reach ncaa tourney time p...,1.276467,2.976021
17201,Casagrande and Golbano unfit for Vuelta Italia...,casagrande golbano unfit vuelta italian cyclis...,-0.323385,0.408005
70437,Gordon repaved Martinsville poses new challen...,gordon repaved martinsville pose new challenge...,0.529955,0.408005


In [133]:
y_val_pred = pipeline.predict(X_val[['processed_text', 'sentiment', 'text_len']])

In [134]:
y_val_pred

array([2, 1, 1, ..., 1, 1, 3])

In [135]:
y_val

,label
112486,2
109163,1
6638,2
3996,2
39522,0
...,...
105468,1
90241,1
17201,1
70437,1


In [136]:
y_train_pred = pipeline.predict(X_train[['processed_text', 'sentiment', 'text_len']])

In [137]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Accuracy
acc = accuracy_score(y_val, y_val_pred)
print("Accuracy:", round(acc, 4))

Accuracy: 0.9099


In [138]:
acc = accuracy_score(y_train, y_train_pred)
print("Accuracy:", round(acc, 4))

Accuracy: 0.9212


In [140]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


In [142]:
pipeline = Pipeline([
    ('features', preprocessor),
    ('clf', LinearSVC())
])

pipeline.fit(X_train[['processed_text', 'sentiment', 'text_len']], y_train)

y_val_pred = pipeline.predict(X_val[['processed_text', 'sentiment', 'text_len']])
y_train_pred = pipeline.predict(X_train[['processed_text', 'sentiment', 'text_len']])


acc = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", round(acc, 4))

acc = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", round(acc, 4))




Train Accuracy: 0.9273
Validation Accuracy: 0.9096


In [147]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('features', preprocessor),
    ('clf', LinearSVC())
])

param_grid = {
    'clf__C': [0.01, 0.1, 1, 5, 10],
    'clf__loss': ['hinge', 'squared_hinge'],
    'clf__max_iter': [1000, 2000]
}

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train[['processed_text', 'sentiment', 'text_len']], y_train)

print("Best parameters:", grid.best_params_)
print("Best cross-val accuracy:", grid.best_score_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best parameters: {'clf__C': 1, 'clf__loss': 'hinge', 'clf__max_iter': 1000}
Best cross-val accuracy: 0.9039895833333333


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [149]:
best_model = grid.best_estimator_

In [150]:
y_val_pred = best_model.predict(X_val[['processed_text', 'sentiment', 'text_len']])

y_train_pred = best_model.predict(X_train[['processed_text', 'sentiment', 'text_len']])

In [151]:
acc = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", round(acc, 4))

acc = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", round(acc, 4))

Train Accuracy: 0.9242
Validation Accuracy: 0.9118


In [161]:


pipeline = Pipeline([
    ('features', preprocessor),
    ('clf', LinearSVC(C=1, loss='hinge', max_iter=1000))])

pipeline.fit(X_train[['processed_text', 'sentiment', 'text_len']], y_train)

y_val_pred = pipeline.predict(X_val[['processed_text', 'sentiment', 'text_len']])
y_train_pred = pipeline.predict(X_train[['processed_text', 'sentiment', 'text_len']])


acc = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", round(acc, 4))

acc = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", round(acc, 4))


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train Accuracy: 0.9242
Validation Accuracy: 0.9118


In [164]:
y_val_pred

array([2, 1, 1, ..., 1, 1, 3])

In [162]:
# Saving the pipeline
joblib.dump(pipeline, "news_topic_pipeline.pkl")

['news_topic_pipeline.pkl']

In [163]:
!cp news_topic_pipeline.pkl /content/drive/MyDrive/data/models/

In [152]:
# # Saving the model to pickle file
# import joblib
# joblib.dump(best_model, "news_topic_classifier_model.pkl")

['news_topic_classifier_model.pkl']

In [157]:
# !cp news_topic_classifier_model.pkl /content/drive/MyDrive/data/models/